In [ ]:
import hashlib
import datetime
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from pymongo import MongoClient
from flask_cors import CORS

#flask app configuration
app = Flask(__name__)
jwt = JWTManager(app)
app.config['JWT_SECRET_KEY'] = 'bagshoo_key'
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = datetime.timedelta(hours=1)
CORS(app)

#database configuration
client = MongoClient("mongodb://localhost:27017/")
db = client["bagshoo"]
users_collection = db["user"]

#register method
@app.route("/restapi/auth/register", methods=["POST"])
def register():
    new_user = request.get_json()#json body request
    new_user["password"] = hashlib.sha256(new_user["password"].encode("utf-8")).hexdigest() #encrpt password
    doc = users_collection.find_one({"email": new_user["email"]}) #check if user exist
    if not doc:
        users_collection.insert_one(new_user)
        return jsonify({'msg': 'User created successfully'}), 200
    else:
        return jsonify({'msg': 'Username already exists'}), 409

#login method
@app.route("/restapi/auth/login", methods=["POST"])
def login():
    login_details = request.get_json() #json body request
    user_from_db = users_collection.find_one({'email': login_details['email']})  #search for user in database

    if user_from_db:
        encrpted_password = hashlib.sha256(login_details['password'].encode("utf-8")).hexdigest()
        if encrpted_password == user_from_db['password']:
            access_token = create_access_token(identity=user_from_db['email']) #create jwt token
            return jsonify(access_token=access_token), 200

    return jsonify({'msg': 'The username or password is incorrect'}), 401

#current user method
@app.route('/restapi/user/email',methods=['GET'])
@jwt_required()
def current_user():
    current_user = get_jwt_identity()# current user details
    return jsonify(logged_in_as=current_user),200

#update name method
@app.route('/restapi/user/update_name',methods=['PUT'])
@jwt_required()
def update_name():
    current_user = get_jwt_identity()# current user details
    update_user = request.get_json()#json body request
    users_collection.update_one({'email': current_user},{'$set': {'name': update_user['name']}})
    return jsonify('updated name successfully'),200

#update phone method
@app.route('/restapi/user/update_phone',methods=['PUT'])
@jwt_required()
def update_phone():
    current_user = get_jwt_identity()# current user details
    update_user = request.get_json()#json body request
    users_collection.update_one({'email': current_user},{'$set': {'phone': update_user['phone']}})
    return jsonify('updated phone number successfully'),200

#update password method
@app.route('/restapi/user/update_password',methods=['PUT'])
@jwt_required()
def update_password():
    current_user = get_jwt_identity()# current user details
    update_user = request.get_json()#json body request
    users_collection.update_one({'email': current_user},{'$set': {'password': hashlib.sha256(update_user['password'].encode("utf-8")).hexdigest()}})
    return jsonify('updated password successfully'),200

#update pincode method
@app.route('/restapi/user/update_pincode',methods=['PUT'])
@jwt_required()
def update_pincode():
    current_user = get_jwt_identity()# current user details
    update_user = request.get_json()#json body request
    users_collection.update_one({'email': current_user},{'$set': {'pincode': update_user['pincode']}})
    return jsonify('updated pincode successfully'),200

#delete method
@app.route('/restapi/user/delete', methods=['DELETE'])
@jwt_required()
def delete_user():
    current_user = get_jwt_identity()# current user details
    users_collection.delete_one({'email': current_user})
    return jsonify('User deleted successfully!'),200

#error handling
@app.errorhandler(404)
def page_not_found():
    message = {
        'status': 404,
        'message': 'Page Not Found: ' + request.url,
    }
    return jsonify(message),404

if __name__ == '__main__':
    app.run(port=3000)